# Ejemplo de uso para morfología de cuencas

Coloque los archivos `dem.tif` y `dir.tif` dentro de `examples/data` antes de ejecutar este cuaderno.

In [ ]:
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from wmf_py.cu_py import basics, metrics

In [ ]:
base = Path("examples/data")
if not base.exists():
    base = Path("data")
dem_path = base / "dem.tif"
dir_path = base / "dir.tif"

with rasterio.open(dem_path) as dem_src:
    dem = dem_src.read(1)
    transform = dem_src.transform
    nodata = dem_src.nodata
with rasterio.open(dir_path) as dir_src:
    flowdir = dir_src.read(1)
xll = transform[2]
yll = transform[5] + transform[4] * dem.shape[0]
dx = transform[0]
dy = -transform[4]
mask = dem != nodata
outlet_x = xll + 3 * dx + dx / 2
outlet_y = yll + 3 * dy + dy / 2
r0, c0 = basics.basin_find(outlet_x, outlet_y, xll, yll, dx, dy, dem.shape[1], dem.shape[0])
basin = basics.basin_cut(dem, (r0, c0), flowdir, mask)
mask_basin = basin["mask"]
acum = basics.basin_acum(flowdir, mask_basin)
stream_mask = acum >= 1
metrics.basin_ppalstream_find(stream_mask, flowdir, dem, dx, dy, (r0, c0))
profile = metrics.basin_ppalstream_cut()
area_km2 = mask_basin.sum() * dx * dy / 1e6
length_km = profile[1, -1] / 1000.0
slope_pct = np.polyfit(profile[1, ::-1], profile[0], 1)[0] * 100

In [ ]:
print(f"Área de la cuenca: {area_km2:.2f} km²")
print(f"Longitud del cauce principal: {length_km:.2f} km")
print(f"Pendiente media del cauce: {slope_pct:.2f} %")

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(dem, cmap="terrain")
plt.contour(mask_basin, levels=[0.5], colors="blue")
plt.title("Cuenca delineada")
plt.axis("off")

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(mask_basin, cmap="Greys", alpha=0.5)
plt.imshow(np.where(stream_mask, stream_mask, np.nan), cmap="Blues", alpha=0.7)
plt.title("Red de drenaje")
plt.axis("off")

In [ ]:
plt.figure()
plt.plot(profile[1], profile[0])
plt.xlabel("Distancia (m)")
plt.ylabel("Elevación (m)")
plt.title("Perfil longitudinal del cauce principal")
plt.grid(True)